In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from xgboost import XGBClassifier

In [2]:
x_train = pd.read_csv('../data/digits/digits_train.data', header=None, delim_whitespace=True)
y_train = (np.genfromtxt('../data/digits/digits_train.labels')+1)/2
x_valid = pd.read_csv('../data/digits/digits_valid.data', header=None, delim_whitespace=True)

In [3]:
x_valid

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,688,0,0,0,0,0,0,952,0,870,...,0,0,0,0,0,0,494,0,769,0
1,778,758,0,0,0,0,0,708,0,991,...,0,0,0,770,0,0,0,0,0,0
2,469,0,816,0,0,0,0,0,0,0,...,0,0,0,467,0,0,0,417,0,0
3,0,0,0,0,0,571,991,983,0,983,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,949,0,991,...,0,0,0,0,0,0,0,0,976,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,511,0,0,0,0,0,0,0,0,...,991,0,0,852,0,0,0,0,0,0
996,0,0,599,0,0,0,0,910,0,0,...,0,0,0,0,0,0,0,0,0,707
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,742
998,0,0,0,0,0,0,0,968,0,0,...,0,0,0,773,0,0,0,0,0,0


In [4]:
def digits_predict(df_train, target_train, df_valid, stat_test):
    s = SelectKBest(stat_test, k=200)
    df_train_sel = s.fit_transform(df_train, target_train)
    df_valid_sel = s.transform(df_valid)
    variables = np.arange(0, len(s.get_support()))[s.get_support()]
    df_train_sel = pd.DataFrame(df_train_sel, columns = variables)
    df_valid_sel = pd.DataFrame(df_valid_sel, columns = variables)
    
    xgb = XGBClassifier(n_estimators=150)
    xgb.fit(df_train_sel, target_train)
    return xgb.predict_proba(df_valid_sel), xgb, variables

In [5]:
preds_chi, model_chi, vars_chi = digits_predict(x_train, y_train, x_valid, chi2)

In [6]:
len(model_chi.feature_importances_) # number of predictors

200

In [7]:
chi_probs = preds_chi[:, 1]

In [8]:
chi_probs.shape

(1000,)

In [9]:
chi_probs

array([8.59902650e-02, 9.63259161e-01, 3.55738448e-04, 9.99897599e-01,
       9.99346077e-01, 5.07601799e-05, 2.11054157e-05, 5.18120505e-05,
       1.07885480e-05, 7.04810100e-06, 9.99406457e-01, 9.99316573e-01,
       4.49516781e-04, 9.99988317e-01, 3.70083787e-02, 9.94917393e-01,
       9.88643706e-01, 9.99966502e-01, 9.99196827e-01, 9.99396801e-01,
       2.27942888e-04, 9.99791324e-01, 9.99496222e-01, 9.99937892e-01,
       9.95494843e-01, 3.75841802e-04, 9.98747826e-01, 9.99576151e-01,
       4.73190472e-03, 8.42136860e-01, 9.97022688e-01, 9.64391589e-01,
       2.34245905e-04, 9.99760926e-01, 9.92935598e-01, 9.99970436e-01,
       9.05360729e-02, 9.63259161e-01, 7.51638261e-04, 4.75330930e-03,
       9.99445021e-01, 7.73401262e-05, 9.63259161e-01, 9.99920368e-01,
       6.44903863e-04, 1.60043861e-03, 9.69675541e-01, 9.35179461e-03,
       9.97837722e-01, 5.82945882e-04, 8.14394025e-06, 1.73245717e-04,
       9.83603968e-06, 6.85601626e-05, 9.99994636e-01, 9.99988317e-01,
      

In [10]:
final_vars = vars_chi + 1

In [11]:
with open('../data/submission/SZYHRY_digits_prediction.txt', 'w') as f:
    f.write('"SZYHRY"\n')
    for c in chi_probs:
        f.write(str(c))
        f.write('\n')

In [12]:
with open('../data/submission/SZYHRY_digits_features.txt', 'w') as f:
    f.write('"SZYHRY"\n')
    for v in final_vars:
        f.write(str(v))
        f.write('\n')

In [13]:
len(final_vars)

200